# LLM for query optimization

The problem is that users sometimes refer only to the product name, as in "Palia" but other times the product name is generic so the company name is included in "Google Keep". If we search for '"Akili Interactive" "EndeavorRx"' we'll filter out a lot of results; searching for "EndeavorRx" is better. But if it's Google Keep we need them both together.



In [11]:
from core import Seed, init

init()

In [14]:
seed = Seed.init("Rad.ai", "Omni", domain="rad.ai")

company_description = "Rad AI is a rapidly growing AI start-up focused on revolutionizing radiology reporting by utilizing advanced machine learning to enhance efficiency, reduce burnout, and improve patient care in healthcare settings."
product_description = "Omni by Rad AI is an intelligent radiology reporting software that automatically generates customized report impressions based on radiologists' findings and preferences, significantly streamlining the reporting process and improving productivity."

queries = [
    f'site:reddit.com "{seed.company}" "{seed.product}"',
    f'site:reddit.com "{seed.product}"',
    # f'"{seed.company}"',
    # f'"{seed.company}" news',
]

queries

['site:reddit.com "Rad.ai" "Omni"', 'site:reddit.com "Omni"']

In [15]:
from utils.google_search import search

queries_results = [list(search(query, num=10)) for query in queries]
queries_results

[[SearchResult(title='Rad AI introduces Rad AI Omni Reporting, an advanced intelligent ...', link='https://www.reddit.com/r/Multiplatform_AI/comments/160h2oo/rad_ai_introduces_rad_ai_omni_reporting_an/', snippet='Aug 24, 2023 ... Rad AI introduces Rad AI Omni Reporting, an advanced intelligent radiology reporting software. r/Multiplatform_AI - Rad AI introduces Rad AI\xa0...', formattedUrl='https://www.reddit.com/r/.../rad_ai_introduces_rad_ai_omni_reporting_an/'),
  SearchResult(title='Powerscribe One -- are you for real? : r/Radiology', link='https://www.reddit.com/r/Radiology/comments/18bymqt/powerscribe_one_are_you_for_real/', snippet='Dec 6, 2023 ... Rad AI is mostly known for automatically generating Impressions and this September launched Omni Reporting which was designed by\xa0...', formattedUrl='https://www.reddit.com/r/Radiology/.../powerscribe_one_are_you_for_real/'),
  SearchResult(title='Is the Grey Matter DNA Asmuths? : r/Ben10', link='https://www.reddit.com/r/Ben10/comme

In [16]:
from typing import Dict, List
from utils.google_search import SearchResult

def index_search_results(search_results: List[SearchResult]) -> Dict[int, SearchResult]:
    indexed_results = {}

    # TODO: Deduplicate search results

    for i, search_result in enumerate(search_results):
        indexed_results[i+1] = search_result

    return indexed_results

def format_search_results(indexed_search_results: Dict[int, SearchResult]) -> str:
    result = ""

    for i, search_result in sorted(indexed_search_results.items()):
        domain = search_result.link.split('/')[2]
        result += f"{i}. {search_result.title} (from {domain})\n"
        result += f"{search_result.snippet}\n\n"

    return result

print(format_search_results(index_search_results(queries_results[0][:5])))

1. Rad AI introduces Rad AI Omni Reporting, an advanced intelligent ... (from www.reddit.com)
Aug 24, 2023 ... Rad AI introduces Rad AI Omni Reporting, an advanced intelligent radiology reporting software. r/Multiplatform_AI - Rad AI introduces Rad AI ...

2. Powerscribe One -- are you for real? : r/Radiology (from www.reddit.com)
Dec 6, 2023 ... Rad AI is mostly known for automatically generating Impressions and this September launched Omni Reporting which was designed by ...

3. Is the Grey Matter DNA Asmuths? : r/Ben10 (from www.reddit.com)
Dec 23, 2023 ... Graymatter/Ben understood what Rad's AI said immediately after transforming ... For all those people who say he'd beat omni man just by becoming a ...




In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field
from typing import List

indexed_search_results = index_search_results(queries_results[0] + queries_results[1])
formatted_search_results = format_search_results(indexed_search_results)

class RatedResult(BaseModel):
    result_number: int = Field(description="Number of the search result", ge=min(indexed_search_results.keys()), le=max(indexed_search_results.keys()))

    customer_relevance: int = Field(description="Relevance of the search result for customers, such as product feedback and critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)
    company_relevance: int = Field(description="Relevance of the search result to the company critique (1 is least relevant, 5 is most relevant)", ge=1, le=5)

class RatedResults(BaseModel):
    rated_queries: List[RatedResult] = Field(description="List of relevance-rated search results")

_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
Review a list of search results and rate their relevance (1-5 scale) to A) the company {company} and B) the customer experience with {product} by {company} on a 1-5 scale.

Additional information:
{company}: {company_description}
{product}: {product_description}
            """,
        ),
        (
            "human",
            """
Search results:
{search_results}
            """,
        ),
    ]
)


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
runnable = _prompt | llm.with_structured_output(RatedResults)
llm_result = runnable.with_config({"run_name": "POC: Rerank search results"}).invoke(
    {
        "company": seed.company,
        "company_description": company_description,
        "product": seed.product,
        "product_description": product_description,
        "search_results": formatted_search_results,
    }
)

llm_result

RatedResults(rated_queries=[RatedResult(result_number=1, customer_relevance=4, company_relevance=5), RatedResult(result_number=2, customer_relevance=4, company_relevance=5), RatedResult(result_number=3, customer_relevance=1, company_relevance=1), RatedResult(result_number=4, customer_relevance=1, company_relevance=1), RatedResult(result_number=5, customer_relevance=1, company_relevance=1), RatedResult(result_number=6, customer_relevance=1, company_relevance=1), RatedResult(result_number=7, customer_relevance=1, company_relevance=1), RatedResult(result_number=8, customer_relevance=1, company_relevance=1), RatedResult(result_number=9, customer_relevance=1, company_relevance=1), RatedResult(result_number=10, customer_relevance=1, company_relevance=1), RatedResult(result_number=11, customer_relevance=1, company_relevance=1), RatedResult(result_number=12, customer_relevance=4, company_relevance=3), RatedResult(result_number=13, customer_relevance=1, company_relevance=1)])

In [18]:
import numpy as np

print(f"""
Number of search results: {len(llm_result.rated_queries)}
Average customer_relevance relevance: {np.mean([rated_result.customer_relevance for rated_result in llm_result.rated_queries]):.1f}
Average company relevance: {np.mean([rated_result.company_relevance for rated_result in llm_result.rated_queries]):.1f}
""")


Number of search results: 13
Average customer_relevance relevance: 1.7
Average company relevance: 1.8



In [19]:
print("""# Customer-relevant search results\n\n""")

for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.customer_relevance, reverse=True)[:10]:
    search_result = indexed_search_results[rated_result.result_number]
    print(f"""
## [{search_result.title}]({search_result.link})
{search_result.snippet}

Customer relevance: {rated_result.customer_relevance}
Company relevance: {rated_result.company_relevance}

""")

print("""# Company-relevant search results\n\n""")
for rated_result in sorted(llm_result.rated_queries, key=lambda x: x.company_relevance, reverse=True)[:10]:
    search_result = indexed_search_results[rated_result.result_number]
    print(f"""
## [{search_result.title}]({search_result.link})
{search_result.snippet}

Customer relevance: {rated_result.customer_relevance}
Company relevance: {rated_result.company_relevance}

""")


# Customer-relevant search results



## [Rad AI introduces Rad AI Omni Reporting, an advanced intelligent ...](https://www.reddit.com/r/Multiplatform_AI/comments/160h2oo/rad_ai_introduces_rad_ai_omni_reporting_an/)
Aug 24, 2023 ... Rad AI introduces Rad AI Omni Reporting, an advanced intelligent radiology reporting software. r/Multiplatform_AI - Rad AI introduces Rad AI ...

Customer relevance: 4
Company relevance: 5



## [Powerscribe One -- are you for real? : r/Radiology](https://www.reddit.com/r/Radiology/comments/18bymqt/powerscribe_one_are_you_for_real/)
Dec 6, 2023 ... Rad AI is mostly known for automatically generating Impressions and this September launched Omni Reporting which was designed by ...

Customer relevance: 4
Company relevance: 5



## [The Downgrade to Omni : r/ChatGPTPro](https://www.reddit.com/r/ChatGPTPro/comments/1cxyxce/the_downgrade_to_omni/)
May 23, 2024 ... 100 votes, 100 comments. I've been remarkably disappointed by Omni since it's drop. While I apprecia